<a href="https://colab.research.google.com/github/masonnlp/question_processing_hacking/blob/master/MedQuAD_QP_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#Download MedQuAD datasets from MedQuAD Github and save in directory
import urllib, os
directory = 'Your/Path/In/Google/Drive'#E.g. directory='/content/gdrive/My Drive/Colab Notebooks/MedQuAD_test/'
os.chdir(directory)
!git clone https://github.com/abachaa/MedQuAD.git

#Install libraries
!pip3 install scispacy #scispacy is able to identiy entities in a question
!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import spacy
import scispacy
import en_core_sci_lg
nlp = en_core_sci_lg.load()
import urllib, os
from lxml import etree as ET
import pandas as pd


In [0]:
def DataReader():
    """
    Read and process questions from MedQuAD dataset in directory
    Write to 'question_processing.xml' file in the directory
    Output format:
      <Input>
        <Question id="0000001-1" doc_id="0000001" source="10_MPlus_ADAM_QA">
          <Q>What is (are) A guide to clinical trials for cancer ?</Q>
          <QP>
            <Category>Disease</Category>
            <Type>information</Type>
            <Theme>A guide to clinical trials for cancer</Theme>
            <NER>clinical trials</NER>
            <NER>cancer</NER>
            <Related type="Synonym">Intervention study</Related>
            <Query>What clinical trials cancer</Query>
            <Query>What A guide to clinical trials for cancer</Query>
            <Query>What Intervention study</Query>
          </QP> 
        </Question>
        ...
      </Input>
    """
    write_root = ET.Element("Input")
    for website_name in [f for f in os.listdir(directory+'MedQuAD/') if f.endswith("QA") or f.endswith("XML")]:
      for xml_file in [f for f in os.listdir(directory + 'MedQuAD/'+ website_name + '/') if f.endswith(".xml")]:
        tree = ET.parse(os.path.join(directory+'MedQuAD/'+ website_name + '/', xml_file))
        root = tree.getroot()
        if root.find('.').get('id') is not None:
          doc_id = root.find('.').get('id')
        else:
          doc_id = root.find('.').get('docid')
        if root.find('./Focus') is not None:
          theme = root.find('./Focus').text
        else: 
          theme = root.find('./doctitle-focus').text
        theme_synonym = root.find('./FocusAnnotations/Synonyms/Synonym')
        if 'MPlus' in website_name:
          category = root.find('./FocusAnnotations/Category').text
        else:  
          category = 'Disease'
        source = website_name
        for qapair in root.findall('./QAPairs/QAPair') or root.findall('./qaPairs/pair'):
          if qapair.find('./Question') is not None:
            qid = qapair.find('./Question')
            question_type = qapair.find('./Question')
            question = qapair.find('Question')
          else:
            qid = qapair.find('./question')
            question_type = qapair.find('./question')
            question = qapair.find('question')
          qid = qid.get('qid')
          question_type = question_type.get('qtype')
          question = question.text
          doc = nlp(question)
          q_qp = ET.SubElement(write_root, "Question")
          q_qp.set('id',qid)
          q_qp.set('doc_id',str(doc_id))
          q_qp.set('source',source)
          q = ET.SubElement(q_qp, "Q")
          q.text = question
          qp = ET.SubElement(q_qp, "QP")
          qcategory = ET.SubElement(qp, "Category")
          qcategory.text = category 
          qtype = ET.SubElement(qp, "Type")
          qtype.text = question_type 
          qtheme = ET.SubElement(qp, "Theme")
          qtheme.text = theme
          entity_list = []
          for ent in doc.ents:
            entity_list.append(str(ent))
            en = ET.SubElement(qp, "NER")
            en.text = str(ent)  
          if theme_synonym is not None:
            synonym = ET.SubElement(qp, "Related")
            synonym.set('type','Synonym')
            synonym.text = theme_synonym.text 
          query = ET.SubElement(qp, "Query")  
          query.text = str(question.strip().split(' ')[0]) + ' ' + ' '.join(entity_list)
          if theme not in entity_list: 
            query = ET.SubElement(qp, "Query")   
            query.text = str(question.strip().split(' ')[0]) + ' ' + str(theme)
          if theme_synonym is not None:
            query = ET.SubElement(qp, "Query")
            query.text = str(question.strip().split(' ')[0]) + ' ' + theme_synonym.text       
    write_tree = ET.ElementTree(write_root)
    write_tree.write(directory + 'question_processing.xml', pretty_print=True)
    

In [0]:
# Read questions from MedQuAD dataset in directory and write to 'question_processing.xml' file in the directory
DataReader()
